In [1]:
'''import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Load datasets
train = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
test = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')
sample_submission = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv')

# Display the first few rows of each DataFrame
print("Train Data Sample:")
print(train.head())

print("\nTest Data Sample:")
print(test.head())

print("\nSample Submission Data Sample:")
print(sample_submission.head())'''


'import pandas as pd\nimport numpy as np\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import RandomizedSearchCV, train_test_split\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.preprocessing import OneHotEncoder\nfrom sklearn.compose import ColumnTransformer\nimport matplotlib.pyplot as plt\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n\n\n# Load datasets\ntrain = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/train.csv\')\ntest = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/test.csv\')\nsample_submission = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv\')\n\n# Display the first few rows of each DataFrame\nprint("Train Data Sample:")\nprint(train.head())\n\nprint("\nTest Data Sam

In [2]:
'''numeric_features = [
    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 
    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'
]

# Plot histograms for the numeric features
train[numeric_features].hist(bins=30, figsize=(20, 15))

# Set the title for the histograms
plt.suptitle('Histograms of Numeric Features')

# Display the histograms
plt.show()'''

"numeric_features = [\n    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', \n    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'\n]\n\n# Plot histograms for the numeric features\ntrain[numeric_features].hist(bins=30, figsize=(20, 15))\n\n# Set the title for the histograms\nplt.suptitle('Histograms of Numeric Features')\n\n# Display the histograms\nplt.show()"

In [3]:
'''import seaborn as sns
# Customize pairplot
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(train[numeric_features], diag_kind='kde', markers='o', palette=None)

# Add title
plt.suptitle('Pairplot of Numeric Features', y=1.02, fontsize=16)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()'''

'import seaborn as sns\n# Customize pairplot\nsns.set(style="ticks", color_codes=True)\ng = sns.pairplot(train[numeric_features], diag_kind=\'kde\', markers=\'o\', palette=None)\n\n# Add title\nplt.suptitle(\'Pairplot of Numeric Features\', y=1.02, fontsize=16)\n\n# Adjust layout\nplt.tight_layout()\n\n# Show plot\nplt.show()'

In [4]:
'''# Set the style of seaborn
sns.set(style="whitegrid")

# Define the figure size
plt.figure(figsize=(10, 6))

# Create box plots for numerical features
sns.boxplot(data=train[numeric_features])

# Set title and labels
plt.title('Box Plot of Numerical Features')
plt.xlabel('Features')
plt.ylabel('Values')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()'''

'# Set the style of seaborn\nsns.set(style="whitegrid")\n\n# Define the figure size\nplt.figure(figsize=(10, 6))\n\n# Create box plots for numerical features\nsns.boxplot(data=train[numeric_features])\n\n# Set title and labels\nplt.title(\'Box Plot of Numerical Features\')\nplt.xlabel(\'Features\')\nplt.ylabel(\'Values\')\n\n# Rotate x-axis labels for better readability\nplt.xticks(rotation=45)\n\n# Show the plot\nplt.show()'

In [5]:
'''import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# Define text column
text_column = 'Status_Description'  # Adjust this if a different column contains text data

# Define text_data_train
text_data_train = train[text_column].fillna('')

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Tokenization (split the text into words)
    tokens = text.split()
    # Remove stop words
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    # Join tokens back into text
    processed_text = ' '.join(tokens)
    return processed_text

# Apply preprocessing to text data
text_data_train_preprocessed = text_data_train.apply(preprocess_text)

# CountVectorizer
count_vectorizer = CountVectorizer(max_features=280, max_df=0.9, min_df=2)

# Fit and transform on preprocessed train data
count_data_train = count_vectorizer.fit_transform(text_data_train_preprocessed)
count_df_train = pd.DataFrame(count_data_train.toarray(), columns=count_vectorizer.get_feature_names_out())

# Compute additional features for train data
word_count = text_data_train.str.split().apply(len)
sentence_count = text_data_train.str.split(r'[.!?]').apply(len)
punctuation_count = text_data_train.str.count(r'[,.!?]')
capitalized_word_count = text_data_train.str.findall(r'\b[A-Z]\w*\b').apply(len)
exclamation_count = text_data_train.str.count(r'!')
question_count = text_data_train.str.count(r'\?')

# Define function to remove outliers
def remove_outliers(data, column, threshold=3):
    """
    Remove outliers from a column of data using the z-score method.
    
    Parameters:
        data (pandas.DataFrame): The DataFrame containing the column.
        column (str): The name of the column from which outliers are to be removed.
        threshold (float): The threshold for considering a data point as an outlier based on its z-score.
                           Default is 3, which is a common threshold.
    
    Returns:
        pandas.DataFrame: The DataFrame with outliers removed from the specified column.
    """
    # Calculate mean and standard deviation of the column
    mean = data[column].mean()
    std_dev = data[column].std()
    
    # Calculate z-scores for each data point in the column
    z_scores = np.abs((data[column] - mean) / std_dev)
    
    # Identify outliers based on z-scores
    outliers = data[z_scores > threshold]
    
    # Remove outliers from the DataFrame
    cleaned_data = data[z_scores <= threshold]
    
    return cleaned_data

# List of numerical features to be used
numeric_features = [
    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 
    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'
]

# Concatenate count features with the original DataFrame for train data
train_final = pd.concat([train[numeric_features], count_df_train, 
                         word_count.rename('WordCount'), 
                         sentence_count.rename('SentenceCount'), 
                         punctuation_count.rename('PunctuationCount'),
                         capitalized_word_count.rename('CapitalizedWordCount'),
                         exclamation_count.rename('ExclamationCount'),
                         question_count.rename('QuestionCount')], axis=1)

# Remove outliers from the numerical features
for feature in numeric_features:
    train_final = remove_outliers(train_final, feature)

# Save train data along with CountVectorizer to a CSV file if needed
train_final.to_csv('train_final.csv', index=False)'''

'import pandas as pd\nimport numpy as np\nimport string\nimport re\nfrom sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.feature_extraction.text import ENGLISH_STOP_WORDS\n# Define text column\ntext_column = \'Status_Description\'  # Adjust this if a different column contains text data\n\n# Define text_data_train\ntext_data_train = train[text_column].fillna(\'\')\n\n# Function to preprocess text\ndef preprocess_text(text):\n    # Convert text to lowercase\n    text = text.lower()\n    # Remove punctuation\n    text = text.translate(str.maketrans(\'\', \'\', string.punctuation))\n    # Remove digits\n    text = re.sub(r\'\\d+\', \'\', text)\n    # Tokenization (split the text into words)\n    tokens = text.split()\n    # Remove stop words\n    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]\n    # Join tokens back into text\n    processed_text = \' \'.join(tokens)\n    return processed_text\n\n# Apply preprocessing to text data\ntext_data_train_

In [6]:
'''# Define text data for testing (assuming 'test' DataFrame is defined)
# Replace 'Recipe_Review' with 'Crime_Category' in the test DataFrame
text_data_test = test['Status_Description'].fillna('')

# Apply preprocessing to text data for testing
text_data_test_preprocessed = text_data_test.apply(preprocess_text)

# Transform test data using the previously fitted CountVectorizer
count_data_test = count_vectorizer.transform(text_data_test_preprocessed)
count_df_test = pd.DataFrame(count_data_test.toarray(), columns=count_vectorizer.get_feature_names_out())

# If you want to save or further process count_df_test, you can do so here
# For example, to save the transformed data to a CSV file:
# count_df_test.to_csv('test_transformed.csv', index=False)'''


"# Define text data for testing (assuming 'test' DataFrame is defined)\n# Replace 'Recipe_Review' with 'Crime_Category' in the test DataFrame\ntext_data_test = test['Status_Description'].fillna('')\n\n# Apply preprocessing to text data for testing\ntext_data_test_preprocessed = text_data_test.apply(preprocess_text)\n\n# Transform test data using the previously fitted CountVectorizer\ncount_data_test = count_vectorizer.transform(text_data_test_preprocessed)\ncount_df_test = pd.DataFrame(count_data_test.toarray(), columns=count_vectorizer.get_feature_names_out())\n\n# If you want to save or further process count_df_test, you can do so here\n# For example, to save the transformed data to a CSV file:\n# count_df_test.to_csv('test_transformed.csv', index=False)"

In [7]:
'''# Define the list of numerical features to include in the final DataFrame
numerical_features = [
    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 
    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'
]

# Ensure the test DataFrame includes all the necessary numerical features
test_final = test[numerical_features].copy()

# Transform the numerical features into a DataFrame
count_data_test = count_vectorizer.transform(text_data_test_preprocessed)
count_df_test = pd.DataFrame(count_data_test.toarray(), columns=count_vectorizer.get_feature_names_out())

# Concatenate the CountVectorizer features with the numerical features
test_final = pd.concat([test_final, count_df_test], axis=1)

# Save the final test DataFrame to a CSV file if needed
test_final.to_csv('test_final.csv', index=False)

# Display the head and shape of the final DataFrame for test data
print(test_final.head())
print(test_final.shape)'''


"# Define the list of numerical features to include in the final DataFrame\nnumerical_features = [\n    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', \n    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'\n]\n\n# Ensure the test DataFrame includes all the necessary numerical features\ntest_final = test[numerical_features].copy()\n\n# Transform the numerical features into a DataFrame\ncount_data_test = count_vectorizer.transform(text_data_test_preprocessed)\ncount_df_test = pd.DataFrame(count_data_test.toarray(), columns=count_vectorizer.get_feature_names_out())\n\n# Concatenate the CountVectorizer features with the numerical features\ntest_final = pd.concat([test_final, count_df_test], axis=1)\n\n# Save the final test DataFrame to a CSV file if needed\ntest_final.to_csv('test_final.csv', index=False)\n\n# Display the head and shape of the final DataFrame for test data\nprint(test_final.head())\nprint(test_final.shape)"

In [8]:
'''# Set the style of seaborn
sns.set(style="whitegrid")

# Plot top Count Vectorizer features
plt.figure(figsize=(15, 10))
sns.barplot(x=count_df_train.columns[:40], y=count_df_train.iloc[:, :40].sum(), palette='viridis')
plt.title('Top 5 Most Common Words (Count Vectorizer)')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# Plot numerical features
# Select numerical features from the DataFrame
numerical_features = [
    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 
    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'
]

# Plot distributions of numerical features
plt.figure(figsize=(20, 15))
for i, feature in enumerate(numerical_features, 1):
    plt.subplot(3, 3, i)
    sns.histplot(train_final[feature].dropna(), kde=True, bins=30, color='blue')
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()'''

'# Set the style of seaborn\nsns.set(style="whitegrid")\n\n# Plot top Count Vectorizer features\nplt.figure(figsize=(15, 10))\nsns.barplot(x=count_df_train.columns[:40], y=count_df_train.iloc[:, :40].sum(), palette=\'viridis\')\nplt.title(\'Top 5 Most Common Words (Count Vectorizer)\')\nplt.xlabel(\'Words\')\nplt.ylabel(\'Frequency\')\nplt.xticks(rotation=45)\nplt.show()\n\n# Plot numerical features\n# Select numerical features from the DataFrame\nnumerical_features = [\n    \'Latitude\', \'Longitude\', \'Time_Occurred\', \'Area_ID\', \n    \'Reporting_District_no\', \'Victim_Age\', \'Premise_Code\', \'Weapon_Used_Code\'\n]\n\n# Plot distributions of numerical features\nplt.figure(figsize=(20, 15))\nfor i, feature in enumerate(numerical_features, 1):\n    plt.subplot(3, 3, i)\n    sns.histplot(train_final[feature].dropna(), kde=True, bins=30, color=\'blue\')\n    plt.title(f\'Distribution of {feature}\')\n    plt.xlabel(feature)\n    plt.ylabel(\'Frequency\')\n\nplt.tight_layout()\nplt

Model 1: Random Forest Classifier

In [9]:
'''from imblearn.pipeline import Pipeline as imPipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Define feature columns and target column
feature_columns = ['Location', 'Cross_Street', 'Latitude', 'Longitude', 'Date_Reported', 'Date_Occurred', 'Time_Occurred', 'Area_ID', 'Area_Name', 'Reporting_District_no', 'Part 1-2', 'Modus_Operandi', 'Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Premise_Code', 'Premise_Description', 'Weapon_Used_Code', 'Weapon_Description', 'Status', 'Status_Description']
target_column = 'Crime_Category'

# Separate features and target
X = train[feature_columns]
y = train[target_column]

# Define preprocessor
numeric_features = [
    'Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 
    'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code'
]
categorical_features = list(set(feature_columns) - set(numeric_features))

numeric_transformer = imPipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = imPipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Define the model with class weights
model = RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=42)

# Adjust hyperparameter grid
param_distributions = {
    'classifier__max_depth': [20, 30, 40],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__bootstrap': [True]
}

# Create pipeline without SMOTE
pipeline = imPipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Perform a train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning with RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=5, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_val)
print('Validation Accuracy:', accuracy_score(y_val, y_pred))
print('Classification Report:')
print(classification_report(y_val, y_pred))'''


"from imblearn.pipeline import Pipeline as imPipeline\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import StandardScaler, OneHotEncoder\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import RandomizedSearchCV, train_test_split\nfrom sklearn.metrics import accuracy_score, classification_report\n\n# Define feature columns and target column\nfeature_columns = ['Location', 'Cross_Street', 'Latitude', 'Longitude', 'Date_Reported', 'Date_Occurred', 'Time_Occurred', 'Area_ID', 'Area_Name', 'Reporting_District_no', 'Part 1-2', 'Modus_Operandi', 'Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Premise_Code', 'Premise_Description', 'Weapon_Used_Code', 'Weapon_Description', 'Status', 'Status_Description']\ntarget_column = 'Crime_Category'\n\n# Separate features and target\nX = train[feature_columns]\ny = train[target_column]\n\n# Define preprocessor\nnumeric_features = [\n    'Latitude', 'Lon

In [10]:
'''# Check the columns in test_final
print("Columns in test_final:", test_final.columns)

# Ensure all required columns are present in test_final
for col in feature_columns:
    if col not in test_final.columns:
        print(f"Warning: Column '{col}' is missing in test_final. Adding it with NaN values.")
        test_final[col] = None  # Add missing columns with NaN values

# Make predictions on the test_final set
# Note: Ensure the test_final DataFrame has all feature columns with the correct names and types
test_final_preprocessed = best_model.named_steps['preprocessor'].transform(test_final[feature_columns])
test_predictions = best_model.named_steps['classifier'].predict(test_final_preprocessed)

# Create a sequential ID column for test_final
test_final['ID'] = test_final.index + 1  # Creates IDs starting from 1

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': test_final['ID'],
    'Crime_Category': test_predictions
})

# Sumission
submission.to_csv('submission.csv', index=False)

# Confirm the file has been saved
print(f"Submission file saved")'''


'# Check the columns in test_final\nprint("Columns in test_final:", test_final.columns)\n\n# Ensure all required columns are present in test_final\nfor col in feature_columns:\n    if col not in test_final.columns:\n        print(f"Warning: Column \'{col}\' is missing in test_final. Adding it with NaN values.")\n        test_final[col] = None  # Add missing columns with NaN values\n\n# Make predictions on the test_final set\n# Note: Ensure the test_final DataFrame has all feature columns with the correct names and types\ntest_final_preprocessed = best_model.named_steps[\'preprocessor\'].transform(test_final[feature_columns])\ntest_predictions = best_model.named_steps[\'classifier\'].predict(test_final_preprocessed)\n\n# Create a sequential ID column for test_final\ntest_final[\'ID\'] = test_final.index + 1  # Creates IDs starting from 1\n\n# Create the submission DataFrame\nsubmission = pd.DataFrame({\n    \'ID\': test_final[\'ID\'],\n    \'Crime_Category\': test_predictions\n})\n\n# S

In [11]:
#submission

Model 2: Decision Tree Classifier

In [12]:
'''import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load datasets
train = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
test = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')
sample_submission = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv')

# Handle missing values
train.fillna({
    'Location': 'Unknown',
    'Cross_Street': 'Unknown',
    'Latitude': train['Latitude'].median(),
    'Longitude': train['Longitude'].median(),
    'Date_Reported': 'Unknown',
    'Date_Occurred': 'Unknown',
    'Time_Occurred': train['Time_Occurred'].median(),
    'Area_ID': train['Area_ID'].median(),
    'Area_Name': 'Unknown',
    'Reporting_District_no': train['Reporting_District_no'].median(),
    'Part 1-2': 'Unknown',
    'Modus_Operandi': 'Unknown',
    'Victim_Age': train['Victim_Age'].median(),
    'Victim_Sex': 'Unknown',
    'Victim_Descent': 'Unknown',
    'Premise_Code': train['Premise_Code'].median(),
    'Premise_Description': 'Unknown',
    'Weapon_Used_Code': train['Weapon_Used_Code'].median(),
    'Weapon_Description': 'Unknown',
    'Status': 'Unknown',
    'Status_Description': 'Unknown'
}, inplace=True)

test.fillna({
    'Location': 'Unknown',
    'Cross_Street': 'Unknown',
    'Latitude': test['Latitude'].median(),
    'Longitude': test['Longitude'].median(),
    'Date_Reported': 'Unknown',
    'Date_Occurred': 'Unknown',
    'Time_Occurred': test['Time_Occurred'].median(),
    'Area_ID': test['Area_ID'].median(),
    'Area_Name': 'Unknown',
    'Reporting_District_no': test['Reporting_District_no'].median(),
    'Part 1-2': 'Unknown',
    'Modus_Operandi': 'Unknown',
    'Victim_Age': test['Victim_Age'].median(),
    'Victim_Sex': 'Unknown',
    'Victim_Descent': 'Unknown',
    'Premise_Code': test['Premise_Code'].median(),
    'Premise_Description': 'Unknown',
    'Weapon_Used_Code': test['Weapon_Used_Code'].median(),
    'Weapon_Description': 'Unknown',
    'Status': 'Unknown',
    'Status_Description': 'Unknown'
}, inplace=True)

# Display the first few rows of each DataFrame
print("Train Data Sample:")
print(train.head())

print("\nTest Data Sample:")
print(test.head())

print("\nSample Submission Data Sample:")
print(sample_submission.head())'''

'import os\nimport pandas as pd\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom sklearn.preprocessing import StandardScaler, OneHotEncoder\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.model_selection import RandomizedSearchCV\n\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# Load datasets\ntrain = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/train.csv\')\ntest = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/test.csv\')\nsample_submission = pd.read_csv(\'/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv\')\n\n# Handle missing values\ntrain.fillna({\n    \'Location\': \'Unknown\',\n    \'Cross_Street\': \'Unknown\',\n    \'Latitude\': train[\'Latitude\'].median(),\

In [13]:
'''# Encode categorical variables
categorical_features = ['Location', 'Cross_Street', 'Date_Reported', 'Date_Occurred', 'Area_Name', 'Part 1-2', 'Modus_Operandi', 'Victim_Sex', 'Victim_Descent', 'Premise_Description', 'Weapon_Description', 'Status', 'Status_Description']
numeric_features = ['Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code']

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Define a simplified param_grid
param_grid = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}'''


"# Encode categorical variables\ncategorical_features = ['Location', 'Cross_Street', 'Date_Reported', 'Date_Occurred', 'Area_Name', 'Part 1-2', 'Modus_Operandi', 'Victim_Sex', 'Victim_Descent', 'Premise_Description', 'Weapon_Description', 'Status', 'Status_Description']\nnumeric_features = ['Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code']\n\n# Create preprocessing pipelines for numeric and categorical data\nnumeric_transformer = Pipeline(steps=[\n    ('scaler', StandardScaler())\n])\n\ncategorical_transformer = Pipeline(steps=[\n    ('onehot', OneHotEncoder(handle_unknown='ignore'))\n])\n\n# Combine preprocessing steps\npreprocessor = ColumnTransformer(\n    transformers=[\n        ('num', numeric_transformer, numeric_features),\n        ('cat', categorical_transformer, categorical_features)\n    ])\n\n# Define the model pipeline\nmodel = Pipeline(steps=[\n    ('preprocessor', preprocessor),\n    ('classifie

In [14]:
'''# Perform Randomized Search with reduced iterations and parallel processing
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(train.drop(columns=['Crime_Category']), train['Crime_Category'])

# Get the best parameters and the best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score:.4f}")

# Train the model with best parameters
best_model = random_search.best_estimator_
best_model.fit(train.drop(columns=['Crime_Category']), train['Crime_Category'])

# Make predictions on the test set
y_pred_test = best_model.predict(test)'''

'# Perform Randomized Search with reduced iterations and parallel processing\nrandom_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring=\'accuracy\', n_jobs=-1, random_state=42)\nrandom_search.fit(train.drop(columns=[\'Crime_Category\']), train[\'Crime_Category\'])\n\n# Get the best parameters and the best score\nbest_params = random_search.best_params_\nbest_score = random_search.best_score_\n\nprint(f"Best parameters: {best_params}")\nprint(f"Best cross-validation score: {best_score:.4f}")\n\n# Train the model with best parameters\nbest_model = random_search.best_estimator_\nbest_model.fit(train.drop(columns=[\'Crime_Category\']), train[\'Crime_Category\'])\n\n# Make predictions on the test set\ny_pred_test = best_model.predict(test)'

In [15]:
'''import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already loaded and preprocessed your dataset as per previous steps

# Define X_train and y_train
X_train = train.drop(columns=['Crime_Category'])
y_train = train['Crime_Category']

# Assuming categorical_features is a list of categorical column names
# One-hot encode categorical features
X_train = pd.get_dummies(X_train, columns=categorical_features)

# Define and train the Decision Tree model with best parameters
dt_classifier = DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=4, max_depth=10, criterion='entropy', random_state=42)
dt_classifier.fit(X_train, y_train)

# Predict on training set
y_train_pred = dt_classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Set Accuracy: {train_accuracy:.4f}")'''


'import pandas as pd\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import accuracy_score\n\n# Assuming you have already loaded and preprocessed your dataset as per previous steps\n\n# Define X_train and y_train\nX_train = train.drop(columns=[\'Crime_Category\'])\ny_train = train[\'Crime_Category\']\n\n# Assuming categorical_features is a list of categorical column names\n# One-hot encode categorical features\nX_train = pd.get_dummies(X_train, columns=categorical_features)\n\n# Define and train the Decision Tree model with best parameters\ndt_classifier = DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=4, max_depth=10, criterion=\'entropy\', random_state=42)\ndt_classifier.fit(X_train, y_train)\n\n# Predict on training set\ny_train_pred = dt_classifier.predict(X_train)\ntrain_accuracy = accuracy_score(y_train, y_train_pred)\nprint(f"Training Set Accuracy: {train_accuracy:.4f}")'

In [16]:
'''# Make predictions on the test set
test_predictions = best_model.predict(test)

# Create a sequential ID column for the test set
test['ID'] = test.index + 1  # Creates IDs starting from 1

# Create a DataFrame with the predictions
submission = pd.DataFrame({
    'ID': test['ID'],
    'Crime_Category': test_predictions
})

# Save the submission file
submission.to_csv('submission.csv', index=False)

# Confirm the file has been saved
print("Submission file saved successfully!")'''


'# Make predictions on the test set\ntest_predictions = best_model.predict(test)\n\n# Create a sequential ID column for the test set\ntest[\'ID\'] = test.index + 1  # Creates IDs starting from 1\n\n# Create a DataFrame with the predictions\nsubmission = pd.DataFrame({\n    \'ID\': test[\'ID\'],\n    \'Crime_Category\': test_predictions\n})\n\n# Save the submission file\nsubmission.to_csv(\'submission.csv\', index=False)\n\n# Confirm the file has been saved\nprint("Submission file saved successfully!")'

In [17]:
#submission

Model 3

In [18]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Load datasets
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
test = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')
sample_submission = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv')

# Print column names for debugging
print("Sample Submission Columns:", sample_submission.columns)
print("Train Columns:", train.columns)
print("Test Columns:", test.columns)

# Feature Engineering: Extracting additional features
def extract_date_features(df):
    df['Date_Reported'] = pd.to_datetime(df['Date_Reported'], errors='coerce')
    df['Date_Occurred'] = pd.to_datetime(df['Date_Occurred'], errors='coerce')
    df['Report_Year'] = df['Date_Reported'].dt.year
    df['Report_Month'] = df['Date_Reported'].dt.month
    df['Report_Day'] = df['Date_Reported'].dt.day
    df['Report_Hour'] = df['Date_Reported'].dt.hour
    df['Occur_Year'] = df['Date_Occurred'].dt.year
    df['Occur_Month'] = df['Date_Occurred'].dt.month
    df['Occur_Day'] = df['Date_Occurred'].dt.day
    df['Occur_Hour'] = df['Date_Occurred'].dt.hour
    df['Time_Difference'] = (df['Date_Reported'] - df['Date_Occurred']).dt.total_seconds() / 3600.0
    df.drop(columns=['Date_Reported', 'Date_Occurred'], inplace=True)
    return df

train = extract_date_features(train)
test = extract_date_features(test)

# Handle missing values
imputer = SimpleImputer(strategy='most_frequent')
train_imputed = pd.DataFrame(imputer.fit_transform(train.drop(columns=['Crime_Category'])), columns=train.columns.drop('Crime_Category'))
train_imputed['Crime_Category'] = train['Crime_Category']
test_imputed = pd.DataFrame(imputer.transform(test), columns=test.columns)

# Encode categorical variables
categorical_features = ['Location', 'Cross_Street', 'Area_Name', 'Part 1-2', 'Modus_Operandi', 'Victim_Sex', 'Victim_Descent', 'Premise_Description', 'Weapon_Description', 'Status', 'Status_Description']
numeric_features = ['Latitude', 'Longitude', 'Time_Occurred', 'Area_ID', 'Reporting_District_no', 'Victim_Age', 'Premise_Code', 'Weapon_Used_Code', 'Report_Year', 'Report_Month', 'Report_Day', 'Report_Hour', 'Occur_Year', 'Occur_Month', 'Occur_Day', 'Occur_Hour', 'Time_Difference']

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(weights='distance', n_neighbors=13, metric='minkowski'))
])

# Split the data into training and validation sets
X = train_imputed.drop(columns=['Crime_Category'])
y = train_imputed['Crime_Category']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate on the validation set
val_predictions = model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Set Accuracy: {val_accuracy:.4f}")


/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv
/kaggle/input/crime-cast-forecasting-crime-categories/train.csv
/kaggle/input/crime-cast-forecasting-crime-categories/test.csv
Sample Submission Columns: Index(['ID', 'Crime_Category ', 'Unnamed: 2'], dtype='object')
Train Columns: Index(['Location', 'Cross_Street', 'Latitude', 'Longitude', 'Date_Reported',
       'Date_Occurred', 'Time_Occurred', 'Area_ID', 'Area_Name',
       'Reporting_District_no', 'Part 1-2', 'Modus_Operandi', 'Victim_Age',
       'Victim_Sex', 'Victim_Descent', 'Premise_Code', 'Premise_Description',
       'Weapon_Used_Code', 'Weapon_Description', 'Status',
       'Status_Description', 'Crime_Category'],
      dtype='object')
Test Columns: Index(['Location', 'Cross_Street', 'Latitude', 'Longitude', 'Date_Reported',
       'Date_Occurred', 'Time_Occurred', 'Area_ID', 'Area_Name',
       'Reporting_District_no', 'Part 1-2', 'Modus_Operandi', 'Victim_Age',
       'Victim_Sex', 'Victim_Descent', 'Premis

/tmp/ipykernel_18/3117387827.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date_Reported'] = pd.to_datetime(df['Date_Reported'], errors='coerce')
/tmp/ipykernel_18/3117387827.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date_Occurred'] = pd.to_datetime(df['Date_Occurred'], errors='coerce')
/tmp/ipykernel_18/3117387827.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date_Reported'] = pd.to_datetime(df['Date_Reported'], errors='coerce')
/tmp/ipykernel_18/3117387827.py:29: UserWarning: Could not infer format, so each element will be parsed indiv

Validation Set Accuracy: 0.7552


In [19]:
# Make predictions on the test set
test_predictions = model.predict(test_imputed)

# Create a sequential ID column for the test set
test['ID'] = test.index + 1  # Creates IDs starting from 1

# Create a DataFrame with the predictions
submission_3 = pd.DataFrame({
    'ID': test['ID'],
    'Crime_Category': test_predictions
})

# Save the submission file
submission_3.to_csv('submission_3.csv', index=False)

# Confirm the file has been saved
print("Submission file saved successfully!")

Submission file saved successfully!


In [20]:
submission_3

,ID,Crime_Category
0,1,Violent Crimes
1,2,Property Crimes
2,3,Violent Crimes
3,4,Property Crimes
4,5,Violent Crimes
...,...,...
4995,4996,Property Crimes
4996,4997,Property Crimes
4997,4998,Violent Crimes
4998,4999,Crimes against Public Order
